## Download the AI-TAC GitHub Repositories

In [ ]:
!git clone https://github.com/smaslova/AI-TAC.git

# Check the contents:
%cd AI-TAC
!ls

Cloning into 'AI-TAC'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 123 (delta 21), reused 15 (delta 15), pack-reused 92 (from 1)
Receiving objects: 100% (123/123), 11.16 MiB | 29.61 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/AI-TAC
code  _config.yml  data_processing  figures  LICENSE  models  README.md  sample_data


In [ ]:
# Install Required Libraries in Google Colab

!pip install scanpy anndata matplotlib seaborn pandas numpy scipy
!pip install tensorflow biopython


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 6.0 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8023 sha256=0672ca1eefead3a0c174843e2de496abaa525a8c54c4180e6628463f0d1c24fc
  Stored in directory: /root/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd998d68a9b268765a
Successfully built session-info
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 59.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement preprocess_utils (from versions: none)
ERROR: No m

In [ ]:
#@title Mount Google Drive

from google.colab import drive

# The following command will prompt a URL for you to click and obtain the
# authorization code

drive.mount("/content/drive/")

# Set data file location
# If you are running notebooks on your laptop, change this to the directory
# where you put downloaded files

from pathlib import Path

DATA = Path("/content/drive/My Drive/ECBM E4060/Final Project Code")

Mounted at /content/drive/


## Load the ImmGen dataset

In [ ]:
import pandas as pd

# Load the datasets
ocrs = pd.read_csv(DATA / "ImmGenATAC18_AllOCRsInfo.csv")
peaks = pd.read_csv(DATA / "ImmGenATAC1219.peak_matched.txt")
normalized_peaks = pd.read_csv(DATA / "mouse_peak_heights.csv")
reference_genome = DATA / "mm10/"  # Path to the reference genome FASTA files (e.g., chr1.fa, chr2.fa, ...)
output_directory = DATA / "Output Data/"


## Preprocess Data

## Preprocess_utils.py code

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import defaultdict
import numpy as np

# takes DNA sequence, outputs one-hot-encoded matrix with rows A, T, G, C
def one_hot_encoder(sequence):
    l = len(sequence)
    x = np.zeros((4,l),dtype = 'int8')
    for j, i in enumerate(sequence):
        if i == "A" or i == "a":
            x[0][j] = 1
        elif i == "T" or i == "t":
            x[1][j] = 1
        elif i == "G" or i == "g":
            x[2][j] = 1
        elif i == "C" or i == "c":
            x[3][j] = 1
        else:
            return "contains_N"
    return x

# read names and positions from bed file
def read_bed(filename):
    positions = defaultdict(list)
    with open(filename) as f:
        for line in f:
            name, chr, start, stop = line.split()
            positions[name].append((chr, int(start), int(stop)))

    return positions

# parse fasta file and turn into dictionary
def read_fasta(genome_dir, num_chr):
    chr_dict = dict()
    for chr in range(1, num_chr):
        chr_file_path = genome_dir + "chr{}.fa".format(chr)
        chr_dict.update(SeqIO.to_dict(SeqIO.parse(open(chr_file_path), 'fasta')))

    return chr_dict

# get sequences for peaks from reference genome
def get_sequences(positions, chr_dict, num_chr):
    one_hot_seqs = []
    peak_seqs = []
    invalid_ids = []
    peak_names = []

    target_chr = [f"Mus_musculus.GRCm39.dna.chromosome.{i}" for i in range(1, num_chr)]

    for name in positions:
        for (chr, start, stop) in positions[name]:
          if chr in target_chr:
                chr_seq = chr_dict[chr].seq
                peak_seq = str(chr_seq)[start - 1:stop].lower()
                one_hot_seq = one_hot_encoder(peak_seq)

                if isinstance(one_hot_seq, np.ndarray):  # it is valid sequence
                    peak_names.append(name)
                    peak_seqs.append(peak_seq)
                    one_hot_seqs.append(one_hot_seq)
                else:
                    invalid_ids.append(name[20:])
          else:
                invalid_ids.append(name[20:])

    one_hot_seqs = np.stack(one_hot_seqs)
    peak_seqs = np.stack(peak_seqs)
    peak_names = np.stack(peak_names)

    return one_hot_seqs, peak_seqs, invalid_ids, peak_names

def format_intensities(intensity_file, invalid_ids):
    cell_type_array = []
    peak_names = []
    with open(intensity_file) as f:
        for i, line in enumerate(f):
            if i == 0: continue
            columns = line.split()
            peak_name = columns[0]
            if '\x1a' not in columns:
                cell_act = columns[1:]
                cell_type_array.append(cell_act)
                peak_names.append(peak_name)

    cell_type_array = np.stack(cell_type_array)
    peak_names = np.stack(peak_names)

    return cell_type_array, peak_names

## Preprocess_data.py Code

In [ ]:
import numpy as np
import json
import os
import pickle as pickle

# Load the datasets
ocrs = pd.read_csv(DATA / "ImmGenATAC18_AllOCRsInfo.csv")
peaks = pd.read_csv(DATA / "ImmGenATAC1219.peak_matched.txt")
normalized_peaks = pd.read_csv(DATA / "mouse_peak_heights.csv")
reference_genome = "/content/drive/My Drive/ECBM E4060/Final Project Code/mm10/"  # Path to the reference genome FASTA files (e.g., chr1.fa, chr2.fa, ...)
output_directory = "/content/drive/MyDrive/ECBM E4060/Final Project Code/data/"
pickle_file = "/content/drive/MyDrive/ECBM E4060/Final Project Code/chr_dict.pickle"

# Create output directory if it doesn't exist
directory = os.path.dirname(output_directory)
if not os.path.exists(directory):
    os.makedirs(directory)

#parameters
num_chr = 20

# read bed file with peak positions, and keep only entries with valid activity vectors
positions = read_bed(DATA / "ImmGenATAC1219.peak_matched.txt")

# Function to read the reference genome fasta files and return a dictionary
def read_fasta(genome_dir, num_chr):
    chr_dict = {}
    for chr_num in range(1, num_chr+1):  # Assume your chromosomes are numbered 1 to num_chr
        chr_file_path = os.path.join(genome_dir, f"Mus_musculus.GRCm39.dna.chromosome.{chr_num}.fa")
        if os.path.exists(chr_file_path):
            chr_dict[f"chr{chr_num}"] = SeqIO.read(chr_file_path, "fasta")  # Read each chromosome
        else:
            print(f"Warning: {chr_file_path} does not exist.")
    return chr_dict

# Check if the pickled reference genome exists; if not, create it
if not os.path.exists(pickle_file):
    chr_dict = read_fasta(reference_genome, num_chr=20)  # Adjust num_chr if necessary
    pickle.dump(chr_dict, open(pickle_file, "wb"))  # Save the dictionary for future use
else:
    # Load the pickled dictionary if it already exists
    chr_dict = pickle.load(open(pickle_file, "rb"))

# Now chr_dict is ready to be used in your code

one_hot_seqs, peak_seqs, invalid_ids, peak_names = get_sequences(positions, chr_dict, num_chr)

# remove invalid ids from intensities file so sequence/intensity files match
cell_type_array, peak_names2 = format_intensities(normalized_peaks, invalid_ids)

cell_type_array = cell_type_array.astype(np.float32)


#take one_hot_sequences of only peaks that have associated intensity values in cell_type_array
peak_ids = np.intersect1d(peak_names, peak_names2)

idx = np.isin(peak_names, peak_ids)
peak_names = peak_names[idx]
one_hot_seqs = one_hot_seqs[idx, :, :]
peak_seqs = peak_seqs[idx]

idx2 = np.isin(peak_names2, peak_ids)
peak_names2 = peak_names2[idx2]
cell_type_array = cell_type_array[idx2, :]

if np.sum(peak_names != peak_names2) > 0:
    print("Order of peaks not matching for sequences/intensities!")

# write to file
np.save(output_directory + 'one_hot_seqs.npy', one_hot_seqs)
np.save(output_directory + 'peak_names.npy', peak_names)
np.save(output_directory + 'peak_seqs.npy', peak_seqs)

with open(output_directory + 'invalid_ids.txt', 'w') as f:
    f.write(json.dumps(invalid_ids))
f.close()

#write fasta file
with open(output_directory + 'sequences.fasta', 'w') as f:
    for i in range(peak_seqs.shape[0]):
        f.write('>' + peak_names[i] + '\n')
        f.write (peak_seqs[i] + '\n')
f.close()

np.save(output_directory + 'cell_type_array.npy', cell_type_array)

ValueError: need at least one array to stack